In [35]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError
from datetime import date, datetime


from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [36]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH = date(2022, 7, 1)

In [37]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [38]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,PwNb0QNNsMmVgoMhjEd7FNNKiNz0iPaOdQsQEBnqkWCVe1xk,8cCpVA1tnAlxNpLVVUJoNPTr-kSLYPQDVrY7O6-CVu8KmI...,r0ezvHEdnAxONYMicm44y-MmQU0XVWnt3HjGRhqOxUWQhT...,I Am Cute AF,7,1656713796000,33,PwNb0QNNsMmVgoMhjEd7FNNKiNz0iPaOdQsQEBnqkWCVe1xk,I Am Cute AF,783,I,107,73,False,False,True,False
1,24xSh26OwD3onDydygWxFqABPPc4uq35QqOTxk3y-lpYJU...,WykjzKc3LAw7nx_6Yewo8tpj8PQC_Wmg4HxhAduFqfoAvu...,6o8dmWsC8KPRIbdoc_odkL2OXQzVpjfJntTSU6v1rvAmLd...,dis rat hiking,29,1656528496000,1,24xSh26OwD3onDydygWxFqABPPc4uq35QqOTxk3y-lpYJU...,dis rat hiking,634,I,92,58,False,False,True,False
2,1jPWoCYQSUgKZeHXT9UIfyXqEYNaDN5vx6LLd8blCDaztpoV,RIzLPt0Yj1Nmx_V91Nq41KkK0jw0N8H6IJTCs3Smthlmc3...,XWwLJnEh-o7p_WMIa2mmoyRKwJoX9bDMucuEsb2PqrSZUK...,Nostereleven,4834,1656840473000,191,1jPWoCYQSUgKZeHXT9UIfyXqEYNaDN5vx6LLd8blCDaztpoV,Nostereleven,635,I,165,125,False,False,False,False
3,E2ameSGs6KV1XKrxoXE0k2yULZrZZGgyA4ZWJ1ORGyzHVCpZ,2YHbKTPpKjEJcGJXr9nHypXKLZmF8s67tyTaElVuGlh933...,ETxLM8791zm7OHyLqr2wvqup8iGY6aOo1ZgSH1j7vwXdse...,inikoiniko,4368,1656845511000,9,E2ameSGs6KV1XKrxoXE0k2yULZrZZGgyA4ZWJ1ORGyzHVCpZ,inikoiniko,1269,I,167,89,False,False,False,True
4,PQKSqNubSJUmfvZEjLVEA2SWGgrrwVlynEEScr15g5Y8JV...,-vbsmRnGhd4x79NQrqpfx4e-VzwchGdnEN5BRY3krQGFtk...,sOtZrQ3l_J1tJeKLTnVpCrcpbKr7d8ukIKDtKcpF1C1cIF...,Artemis Simp,5374,1656865170000,19,PQKSqNubSJUmfvZEjLVEA2SWGgrrwVlynEEScr15g5Y8JV...,Artemis Simp,588,I,287,240,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,v0V8za9lUe9Sal3pWMudPbeLKAV9KJam1IhcUktvoCpwFQ,FjLKIVYYUSrV37F_ZcBfkdPfRtkqXlYgw5u8WWv6WI7MYX...,zHydration,585,1656812123000,37,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,zHydration,781,I,120,70,False,False,True,False
135,Y298V36HCyTqMQF4l3arT7Hn74Z8VgW-MJPbRYYVCeAEzjM,hw_VRXwFk5qe2lbGMuM-pXdIROKiUyuvpal2YxPh8kJptw,q6g9BjTYbn3Py1-YQndWy0_tzJV45lk7_CBu_w2oafUuc-...,Aesah,937,1656835090000,120,Y298V36HCyTqMQF4l3arT7Hn74Z8VgW-MJPbRYYVCeAEzjM,Aesah,1500,I,151,82,False,False,False,True
136,wQWLnBQkUum2eZZ_4DVYe_rKJAl8lj0q6zbUzn0KuCVvjkRO,hHaThCC-vyrN9u1KAXno3hEeGn5n4K6jIE9iDzt6XJLSq7...,Dn-dVJuWD7zOCxzVdYBplq2bC0y3Vl3aUxPLcTbB9tX_Zc...,VX15375231123,4379,1656823460000,30,wQWLnBQkUum2eZZ_4DVYe_rKJAl8lj0q6zbUzn0KuCVvjkRO,VX15375231123,649,I,109,69,False,False,True,True
137,rseOMGMOy6sMmC_BEUN8Rrzur0bedb1TyOJbKvdxThJtAhE,W3cdKoMwYdbDhIFn_7ZYvdTZmXpcdO6UzedkNu_TDBppjPY,lLtsw7rM-yW8RierLNxTKjpGlfn01KJr4Jdovj-7jqNenj...,Ramblinnn,3591,1656712261000,12,rseOMGMOy6sMmC_BEUN8Rrzur0bedb1TyOJbKvdxThJtAhE,Ramblinnn,655,I,108,70,False,False,False,False


In [39]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [40]:
len(uniq_matches_id)

4361

In [41]:
len(matches_asset)

7351

In [42]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [43]:
len(uniq_matches)

4361

In [44]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [45]:
latest_matches = [match for match in uniq_matches if (
    LATEST_RELEASE in match['info']['game_version'])]


In [46]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [47]:
len(latest_matches)

2693

In [48]:
len(latest_patch_matches)

912

In [49]:
# latest_matches[0]['info']

In [50]:
from typing import List


def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                if augment == 'TFT7_Augment_GuildLootHR':
                    augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item

            matches_array.append(match)

    return matches_array


In [51]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [52]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4360081160',
 'placement': 8,
 'augment0': 'TFT7_Augment_JadeEmblem',
 'augment1': 'TFT6_Augment_FirstAidKit',
 'augment2': 'TFT6_Augment_CyberneticImplants2',
 'Set7_Bard': 1,
 'Set7_Dragon': 1,
 'Set7_Dragonmancer': 0,
 'Set7_Evoker': 0,
 'Set7_Guild': 1,
 'Set7_Jade': 3,
 'Set7_Legend': 0,
 'Set7_Mystic': 1,
 'Set7_Shapeshifter': 1,
 'Set7_Starcaller': 1,
 'Set7_Swiftshot': 0,
 'TFT7_Gnar': 1,
 'TFT7_Ashe': 1,
 'TFT7_Ashe_item0': 'TFT_Item_ThiefsGloves',
 'TFT7_Ashe_item1': 'TFT_Item_Zephyr',
 'TFT7_Ashe_item2': 'TFT_Item_GuardianAngel',
 'TFT7_Anivia': 2,
 'TFT7_Anivia_item0': 'TFT_Item_NeedlesslyLargeRod',
 'TFT7_Neeko': 1,
 'TFT7_Neeko_item0': 'TFT_Item_RedBuff',
 'TFT7_Neeko_item1': 'TFT_Item_GargoyleStoneplate',
 'TFT7_Neeko_item2': 'TFT_Item_WarmogsArmor',
 'TFT7_DragonGreen': 1,
 'TFT7_DragonGreen_item0': 'TFT_Item_Quicksilver',
 'TFT7_DragonGreen_item1': 'TFT_Item_MadredsBloodrazor',
 'TFT7_Bard': 1,
 'TFT7_Bard_item0': 'TFT7_Item_JadeEmblemItem',
 'TFT7_So

In [53]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [54]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bard,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,Set7_Guild,...,TFT7_Gnar_item2,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Sett_item2,TFT7_TahmKench_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4360081160,8,TFT7_Augment_JadeEmblem,TFT6_Augment_FirstAidKit,TFT6_Augment_CyberneticImplants2,1.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4360081160,1,TFT6_Augment_SecondWind1,TFT6_Augment_PandorasItems,TFT6_Augment_Distancing2,1.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4360081160,6,TFT6_Augment_TinyTitans,TFT7_Augment_BruiserTrait,TFT7_Augment_BigFriend2,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4360081160,3,TFT7_Augment_JadeEmblem,TFT6_Augment_Diversify1,TFT7_Augment_JadeEternalProtection,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4360081160,2,TFT6_Augment_CelestialBlessing1,TFT7_Augment_BestFriends1,TFT6_Augment_Diversify2,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21539,NA1_4355845757,5,TFT6_Augment_PortableForge,TFT6_Augment_TinyTitans,TFT6_Augment_CelestialBlessing2,1.0,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21540,NA1_4355845757,3,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_TinyTitans,TFT6_Augment_Ascension,NaN,1.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21541,NA1_4355845757,7,TFT6_Augment_PortableForge,TFT6_Augment_Diversify1,TFT6_Augment_CyberneticShell2,1.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21542,NA1_4355845757,1,TFT6_Augment_CelestialBlessing2,TFT6_Augment_Electrocharge1,TFT6_Augment_JeweledLotus,1.0,NaN,0.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bard,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,Set7_Guild,...,TFT7_Gnar_item2,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Sett_item2,TFT7_TahmKench_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4360081160,8,TFT7_Augment_JadeEmblem,TFT6_Augment_FirstAidKit,TFT6_Augment_CyberneticImplants2,1.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4360081160,1,TFT6_Augment_SecondWind1,TFT6_Augment_PandorasItems,TFT6_Augment_Distancing2,1.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4360081160,6,TFT6_Augment_TinyTitans,TFT7_Augment_BruiserTrait,TFT7_Augment_BigFriend2,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4360081160,3,TFT7_Augment_JadeEmblem,TFT6_Augment_Diversify1,TFT7_Augment_JadeEternalProtection,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4360081160,2,TFT6_Augment_CelestialBlessing1,TFT7_Augment_BestFriends1,TFT6_Augment_Diversify2,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21539,NA1_4355845757,5,TFT6_Augment_PortableForge,TFT6_Augment_TinyTitans,TFT6_Augment_CelestialBlessing2,1.0,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21540,NA1_4355845757,3,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_TinyTitans,TFT6_Augment_Ascension,NaN,1.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21541,NA1_4355845757,7,TFT6_Augment_PortableForge,TFT6_Augment_Diversify1,TFT6_Augment_CyberneticShell2,1.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21542,NA1_4355845757,1,TFT6_Augment_CelestialBlessing2,TFT6_Augment_Electrocharge1,TFT6_Augment_JeweledLotus,1.0,NaN,0.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [57]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

In [58]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)